<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Celdas_Asentamiento_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: FREATIMETROS (XLSX)


,FECHA,FREATIMETRO,COTA_RÌO_(M.S.N.M),PROF_NIVEL_FREATICO_(M),COTA_NIVEL_FREATICO_(M.S.N.M.),CARGA_(M.C.A),PRECIPITACIONES
0,09/11/2020,FR-0+775,NaN,5.273,114.158,10.407,1.8
1,12/11/2020,FR-0+775,NaN,5.181,114.250,10.499,0.0
2,04/12/2020,FR-0+775,115.868,4.658,114.773,11.022,0.0
3,23/01/2021,FR-0+775,116.540,3.680,115.751,12.000,0.0
4,31/01/2021,FR-0+775,116.540,3.560,115.871,12.120,0.0


In [3]:
# === VISUALIZACIÓN ASENTAMIENTO CON SELECTOR DE ORIGEN, PROGRESIVA Y CELDA ===

import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# --- Widgets y salidas ---
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
celda_dropdown = widgets.Dropdown(description="Celda:")
punto_fijo_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)",
    description="Estilo:"
)

tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500),
    description="Tamaño:"
)

grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2,
    description="Grosor:"
)

paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly,
    description="Paleta:"
)

output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_asentamiento", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png",
    description="Formato:"
)

controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# --- Función para obtener DataFrame según origen ---
def obtener_df(origen):
    try:
        return datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
    except:
        return pd.DataFrame()

# --- Limpiar nombres de columnas ---
def limpiar_columnas(df):
    """Limpia nombres de columnas eliminando espacios y caracteres especiales"""
    if df.empty:
        return df

    # Mapeo de nombres posibles a nombres estándar
    mapeo_columnas = {
        'FECHA': 'FECHA',
        'CELDA_DE_ASENTAMIENTO': 'CELDA_DE_ASENTAMIENTO',
        'PROGRESIVA': 'PROGRESIVA',
        'PUNTO_FIJO_CASETA': 'PUNTO_FIJO_CASETA',
        'COTA_RIO_(M.S.N.M)': 'COTA_RIO_MSNM',
        'COTA_RIO_MSNM': 'COTA_RIO_MSNM',
        'LECTURA_REGLETA_(M)': 'LECTURA_REGLETA_M',
        'LECTURA_REGLETA_M': 'LECTURA_REGLETA_M',
        'COTA_0_REGLETA_(M)': 'COTA_0_REGLETA_M',
        'COTA_0_REGLETA_M': 'COTA_0_REGLETA_M',
        'COTA_CELDA_(M.S.N.M)': 'COTA_CELDA_MSNM',
        'COTA_CELDA_MSNM': 'COTA_CELDA_MSNM',
        'ASENTAMIENTO_(CM)': 'ASENTAMIENTO_CM',
        'ASENTAMIENTO_CM': 'ASENTAMIENTO_CM',
        'COTA_RELLENO': 'COTA_RELLENO',
        'MODULO_DEFORMACION_(Î•)': 'MODULO_DEFORMACION',
        'MODULO_DEFORMACION': 'MODULO_DEFORMACION'
    }

    # Renombrar columnas
    df_clean = df.copy()
    for col_original, col_nueva in mapeo_columnas.items():
        if col_original in df_clean.columns:
            df_clean = df_clean.rename(columns={col_original: col_nueva})

    return df_clean

# --- Actualizar PROGRESIVAS disponibles ---
def actualizar_progresivas(change=None):
    df = obtener_df(origen_dropdown.value)
    df = limpiar_columnas(df)
    if df.empty or 'PROGRESIVA' not in df.columns:
        progresiva_dropdown.options = []
        return
    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown.options = progresivas
    if progresivas:
        progresiva_dropdown.value = progresivas[0]

# --- Actualizar CELDAS disponibles según PROGRESIVA ---
def actualizar_celdas(change=None):
    df = obtener_df(origen_dropdown.value)
    df = limpiar_columnas(df)
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    prog = progresiva_dropdown.value
    if prog:
        celdas = sorted(df[df['PROGRESIVA'] == prog]['CELDA_DE_ASENTAMIENTO'].dropna().unique())
        celda_dropdown.options = ["Todas"] + list(celdas)
        celda_dropdown.value = "Todas"

# --- Actualizar PUNTOS FIJOS disponibles ---
def actualizar_puntos_fijos(change=None):
    df = obtener_df(origen_dropdown.value)
    df = limpiar_columnas(df)
    if df.empty or 'PUNTO_FIJO_CASETA' not in df.columns:
        punto_fijo_dropdown.options = []
        return
    puntos = sorted(df['PUNTO_FIJO_CASETA'].dropna().unique())
    punto_fijo_dropdown.options = ["Todos"] + list(puntos)
    punto_fijo_dropdown.value = "Todos"

# --- Actualizar VARIABLES y AÑOS ---
def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    df = limpiar_columnas(df)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return

    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

    # Variables disponibles para graficar (numéricas principales)
    columnas_excluir = ['FECHA', 'CELDA_DE_ASENTAMIENTO', 'PROGRESIVA', 'PUNTO_FIJO_CASETA']
    variables_numericas = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]

    # Variables principales esperadas
    variables_principales = [
        'COTA_RIO_MSNM', 'LECTURA_REGLETA_M', 'COTA_0_REGLETA_M',
        'COTA_CELDA_MSNM', 'ASENTAMIENTO_CM', 'COTA_RELLENO', 'MODULO_DEFORMACION'
    ]

    # Combinar variables detectadas con las principales
    variables = list(set(variables_numericas + [v for v in variables_principales if v in df.columns]))

    variable_dropdown.options = variables
    if variables:
        # Priorizar ASENTAMIENTO_CM si está disponible
        if 'ASENTAMIENTO_CM' in variables:
            variable_dropdown.value = 'ASENTAMIENTO_CM'
        else:
            variable_dropdown.value = variables[0]

    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()
        df = limpiar_columnas(df)

        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

        prog = progresiva_dropdown.value
        celda = celda_dropdown.value
        punto_fijo = punto_fijo_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        # Filtrar datos
        df_plot = df[df['PROGRESIVA'] == prog]

        if celda != "Todas":
            df_plot = df_plot[df_plot['CELDA_DE_ASENTAMIENTO'] == celda]

        if punto_fijo != "Todos":
            df_plot = df_plot[df_plot['PUNTO_FIJO_CASETA'] == punto_fijo]

        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]

        df_plot = df_plot.dropna(subset=['FECHA', variable])

        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()

        # Determinar cómo agrupar las series
        if celda == "Todas":
            # Graficar todas las celdas como series separadas
            celdas = sorted(df_plot['CELDA_DE_ASENTAMIENTO'].unique())
            color_map = {c: paleta[i % len(paleta)] for i, c in enumerate(celdas)}

            for celda_actual in celdas:
                datos_celda = df_plot[df_plot['CELDA_DE_ASENTAMIENTO'] == celda_actual]
                line_args = dict(width=grosor, color=color_map[celda_actual])
                marker_args = dict(color=color_map[celda_actual])

                modo = {
                    "Curvas suaves (spline)": ("lines", "spline"),
                    "Líneas rectas": ("lines", "linear"),
                    "Puntos": ("markers", None),
                    "Líneas + Puntos": ("lines+markers", "linear"),
                    "Área apilada": ("lines", "linear"),
                    "Área + Líneas": ("lines", "linear"),
                    "Área + Líneas + Puntos": ("lines+markers", "linear")
                }
                modo_graf, line_shape = modo[estilo]
                fill = "tozeroy" if "Área" in estilo else None
                stackgroup = "one" if estilo == "Área apilada" else None

                fig.add_trace(go.Scatter(
                    x=datos_celda['FECHA'],
                    y=datos_celda[variable],
                    mode=modo_graf,
                    name=f"Celda {celda_actual}",
                    line_shape=line_shape,
                    line=line_args,
                    marker=marker_args,
                    fill=fill,
                    stackgroup=stackgroup
                ))
        else:
            # Graficar una sola celda
            line_args = dict(width=grosor, color=paleta[0])
            marker_args = dict(color=paleta[0])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None

            fig.add_trace(go.Scatter(
                x=df_plot['FECHA'],
                y=df_plot[variable],
                mode=modo_graf,
                name=f"Prog. {prog} - Celda {celda}",
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill
            ))

        # Configurar layout
        titulo = f"Progresiva {prog} – {variable}"
        if celda != "Todas":
            titulo += f" (Celda {celda})"
        if punto_fijo != "Todos":
            titulo += f" - P.F. {punto_fijo}"

        # Determinar unidad para el eje Y según la variable
        unidad_y = ""
        if "CM" in variable.upper():
            unidad_y = " (cm)"
        elif "MSNM" in variable.upper() or "_M" in variable.upper():
            unidad_y = " (m)"
        elif "MODULO" in variable.upper():
            unidad_y = " (ε)"

        fig.update_layout(
            width=ancho, height=alto,
            title=titulo,
            xaxis_title="Fecha",
            yaxis_title=variable + unidad_y,
            legend_title="Celda" if celda == "Todas" else "Serie",
            hovermode="x unified"
        )
        fig.show()

# --- Guardar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# --- Eventos ---
origen_dropdown.observe(actualizar_progresivas, names='value')
progresiva_dropdown.observe(actualizar_celdas, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')
origen_dropdown.observe(actualizar_puntos_fijos, names='value')

boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Inicializar y mostrar controles ---
actualizar_progresivas()
actualizar_variables_y_anios()
actualizar_celdas()
actualizar_puntos_fijos()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Asentamiento</h2>"))
display(origen_dropdown)
display(widgets.HBox([progresiva_dropdown, celda_dropdown]))
display(widgets.HBox([punto_fijo_dropdown, variable_dropdown]))
display(widgets.HBox([anio_dropdown, estilo_dropdown]))
display(widgets.HBox([tamanio_dropdown, grosor_dropdown]))
display(paleta_dropdown)
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)

Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Dropdown(description='Paleta:', options={'Plotly': ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#1…

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()